In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
datagen = ImageDataGenerator(
    rescale=1./255,  # normalize pixel values to [0,1]
    rotation_range=20,  # randomly rotate images
    width_shift_range=0.2,  # randomly shift images horizontally
    height_shift_range=0.2,  # randomly shift images vertically
    shear_range=0.2,  # randomly apply shear transformation
    zoom_range=0.2,  # randomly zoom in and out on images
    horizontal_flip=True,  # randomly flip images horizontally
    fill_mode='nearest'  # fill in newly created pixels
)

def get_label(filename):
    if filename.startswith("homer"):
        return 'homer'
    else:
        return 'bart'

image_filenames = os.listdir('homer_bart_1')
labels = [get_label(filename) for filename in image_filenames]

train_filenames, test_filenames, train_labels, test_labels = train_test_split(image_filenames, labels, test_size=0.2, random_state=42)


train_df = pd.DataFrame({"filename": train_filenames, "label": train_labels})
test_df = pd.DataFrame({"filename": test_filenames, "label": test_labels})


train_generator = datagen.flow_from_dataframe(
    train_df,
    directory='homer_bart_1',
    x_col="filename",
    y_col="label",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_dataframe(
    test_df,
    directory='homer_bart_1',
    x_col="filename",
    y_col="label",
    target_size=(150, 150), 
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42
)

Found 215 validated image filenames belonging to 2 classes.
Found 54 validated image filenames belonging to 2 classes.


In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 17, 17, 128)      

In [12]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
)

Epoch 1/20
7/7 [==============================] - 6s 702ms/step - loss: 1.5972 - accuracy: 0.5163 - val_loss: 0.6794 - val_accuracy: 0.5741
Epoch 2/20
7/7 [==============================] - 4s 623ms/step - loss: 0.6695 - accuracy: 0.6047 - val_loss: 0.6818 - val_accuracy: 0.5741
Epoch 3/20
7/7 [==============================] - 4s 637ms/step - loss: 0.7014 - accuracy: 0.6233 - val_loss: 0.6607 - val_accuracy: 0.5741
Epoch 4/20
7/7 [==============================] - 4s 625ms/step - loss: 0.6471 - accuracy: 0.6000 - val_loss: 0.6613 - val_accuracy: 0.5741
Epoch 5/20
7/7 [==============================] - 4s 659ms/step - loss: 0.6337 - accuracy: 0.6000 - val_loss: 0.6611 - val_accuracy: 0.5741
Epoch 6/20
7/7 [==============================] - 5s 641ms/step - loss: 0.6000 - accuracy: 0.6047 - val_loss: 0.6279 - val_accuracy: 0.5741
Epoch 7/20
7/7 [==============================] - 4s 633ms/step - loss: 0.5507 - accuracy: 0.7023 - val_loss: 0.5674 - val_accuracy: 0.7037
Epoch 8/20
7/7 [====